In [8]:
import math
train_list='train'
test_list='test'
out_list='outputlist'
hamwordcnt = 0
spamwordcnt = 0
summail = 0
wordcnt = {}
mailcnt = {}
prior = {} 
conditional_list = {} 

vocabolary_list = set()

def initialize():
    global summail
    global hamwordcnt
    global spamwordcnt
    global mailcnt
    global wordcnt
    global prior
    global conditional_list
    global vocabolary_list

    summail = 0
    hamwordcnt = 0
    spamwordcnt = 0
    mailcnt = {}
    wordcnt = {}
    prior = {}
    conditional_list = {}
    vocabolary_list = set()

    trainer_list = open(train_list,'r')
    

    for line in trainer_list:
        voca=[]
        take_data = line.split(" ")[2:] 
        for (index, word) in enumerate(take_data):
            if (index % 2) == 0:
                voca += [word]

        for it in voca:
            vocabolary_list.add(it)

    trainer_list.close()
def calculate_prior_conditional_prob():
    global vocabolary_list
    global summail
    global mailcnt
    global wordcnt
    global prior
    global conditional_list
    global hamwordcnt
    global spamwordcnt
    wordcnt.setdefault("spam", {})
    wordcnt.setdefault("ham", {})

    for word in vocabolary_list:
        wordcnt["spam"].setdefault(word, 0)
        wordcnt["ham"].setdefault(word, 0)

    trainer_list = open(train_list, 'r')

    for line in trainer_list:
        take_data = line.split(" ")
        class_type = take_data[1]
        mailcnt.setdefault(class_type, 0)
        mailcnt[class_type] += 1
        summail += 1
        for i in range(2, len(take_data), 2):
            wordcnt[class_type][take_data[i]] += int(take_data[i + 1])

    for word, count_data in wordcnt["ham"].items():
        hamwordcnt += count_data
    for word, count_data in wordcnt["spam"].items():
        spamwordcnt += count_data

    for class_type, count_data in mailcnt.items():
        prior.setdefault(class_type, 0.0)
        prior[class_type] = float(count_data / summail)
        print(class_type,end='')
        print(" prior probability: "+str(float(count_data/summail)))
    conditional_list.setdefault("ham", {})
    conditional_list.setdefault("spam", {})

    for class_type, item in wordcnt.items():
        for word, count_data in item.items():
            conditional_list[class_type].setdefault(word, 0)
            if (class_type in "spam"):
                conditional_list[class_type][word] = float(
                    (count_data + 1) / float(spamwordcnt + len(vocabolary_list)))
            elif (class_type in "ham"):
                conditional_list[class_type][word] = float(
                    (count_data + 1) / float(hamwordcnt + len(vocabolary_list)))


def testing_by_given_data():
    global conditional_list
    global hamwordcnt
    global spamwordcnt

    testing_data = open(test_list,'r')
    output = open(out_list,'w')

    actual_class_list = list()
    class_list_after_calculated = list()
    for line_ in testing_data:
        take_data = line_.split(" ")
        test_mail_id = take_data[0]
        actual_class_data = take_data[1]

        predict_class = ""
        collect_word = []
        for index, word in enumerate(take_data[2:]):
            if index % 2 == 0:
                collect_word += [word]

        ham_probability = 0.0
        spam_probability = 0.0
        for word in collect_word:
            if word not in vocabolary_list:
                ham_probability += math.log10(1 / float(hamwordcnt + len(vocabolary_list)))
                spam_probability += math.log10(1 / float(spamwordcnt + len(vocabolary_list)))
            else:
                ham_probability += math.log10(conditional_list["ham"][word])
                spam_probability += math.log10(conditional_list["spam"][word])

        if spam_probability > ham_probability:
            predict_class = "spam"
        else:
            predict_class = "ham"

        actual_class_list.append(actual_class_data)
        class_list_after_calculated.append(predict_class)
        output.write(test_mail_id + " " + predict_class + "\n")

    testing_data.close()
    output.close()

    compareclass(actual_class_list, class_list_after_calculated)

def compareclass(actualtrue_list, predicttrue_list):

    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0


    for actualdata,predictdata in zip(actualtrue_list,predicttrue_list):
        if (predictdata in "spam"):
            if(actualdata in "spam"):
                true_positive += 1
            elif(actualdata in "ham"):
                false_positive += 1

        elif (predictdata in "ham"):
            if(actualdata in "ham"):
                true_negative += 1
            elif (actualdata in "spam"):
                false_negative += 1

    precision = float(true_positive/float(true_positive+false_positive)) * 100
    recall = float(true_positive/float(true_positive+false_negative)) * 100
    f1score = 2*(precision*recall)/(recall+precision)
    print("precision= ",end='')
    print(precision,end='')
    print("%")
    print("recall= ",end='')
    print(recall,end='')
    print("%")
    print("F1-Score= ",end='')
    print(f1score,end='')
    print("%")

if __name__ == "__main__":

     initialize()
      
     calculate_prior_conditional_prob()
    
     testing_by_given_data()

ham prior probability: 0.42633333333333334
spam prior probability: 0.5736666666666667
precision= 89.35483870967742%
recall= 95.51724137931035%
F1-Score= 92.33333333333333%
